In [1]:
import requests
import json
import sqlalchemy
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
con = sqlalchemy.create_engine('postgresql://postgres:@127.0.0.1:8181/lognex')

In [ ]:
# Изменяемые параметры:

namespace = 6  # Неймспейс
ticket_number = 1  # Количество заявок на 1 поток
num_threads = 1  # Количество потоков

# Запрос
url = f'https://subzero-billing-{namespace}.testms-test.lognex.ru/api/clinton/1.0/ticket'
headers = {'Content-Type': 'application/json'}

In [4]:
# Фукнция для запроса к базе

def sql_go(request):
    return pd.read_sql(sqlalchemy.text(request), con)

In [2]:
sql_product = '''
            select name as "Название_продукта", internal_id as "ID_продукта", trialtariff_id as "ID_триального_тарифа"
from billing.productversion
where name ILIKE 'Test_product_%'
        '''

In [24]:
product_list = sql_go(sql_product)
product_list = product_list.set_index('Название_продукта').reset_index()
product_list

,Название_продукта,ID_продукта,ID_триального_тарифа
0,Test_product_1,71375ff5-972a-11ed-0a82-072200000038,b313d76d-290b-4be8-9e64-f24dddb7bd2e
1,Test_product_2,71bc7c4c-972a-11ed-0a82-07220000003e,fd2be12d-16fc-4cc1-b2c9-6cd3342665db
2,Test_product_3,722d9d51-972a-11ed-0a82-072200000044,1019ec92-379e-49cc-8517-b624ef6d711d
3,Test_product_4,72a7c7ee-972a-11ed-0a82-07220000004a,ae6b7f43-dd5e-47bf-b807-ec9247ad05db
4,Test_product_5,732d6311-972a-11ed-0a82-072200000050,15af65fa-ef18-44d8-84e7-f116ac13176b
5,Test_product_6,73a10b67-972a-11ed-0a82-072200000056,f95c7a27-26af-40f2-a8e5-dde0be0dbcc4


In [23]:
test_id = product_list.loc['Test_product_1', 'ID_триального_тарифа']
test_id

UUID('b313d76d-290b-4be8-9e64-f24dddb7bd2e')

In [19]:
for index, row in product_list.iterrows():
    print (row['ID_триального_тарифа'])

b313d76d-290b-4be8-9e64-f24dddb7bd2e
fd2be12d-16fc-4cc1-b2c9-6cd3342665db
1019ec92-379e-49cc-8517-b624ef6d711d
ae6b7f43-dd5e-47bf-b807-ec9247ad05db
15af65fa-ef18-44d8-84e7-f116ac13176b
f95c7a27-26af-40f2-a8e5-dde0be0dbcc4


In [8]:
account_list = sql_go('''select id from billing.billingaccount WHERE company LIKE 'test%' ''')
account_list

,id
0,2a86c4fb-9710-11ed-0a82-07220000000c
1,538cb9a4-964a-11ed-0a82-072200000002


In [29]:
print('bomber start')

counter = 0

with requests.Session() as s:
    with open('json1.json', 'r') as json1:
        sub_data = json.load(json1)
        for i in range(ticket_number):
            for account_id in account_list:
                sub_data['accountId'] = account_id

                for index, row in product_list.iterrows():
                    
                    sub_data['product']['id'] = product_list.loc[index, 'ID_продукта']
                    sub_data['tariff']['id'] = product_list.loc[index, 'ID_триального_тарифа']
        
                    counter += 1
                    print(sub_data)
                    print('--------------------------------------------------')
                    # s.post(url, data=json.dumps(sub_data), headers=headers)
                    # print('', counter1)

    print('------------------------------------------------')
    print('итого, создано заявок:', counter)

bomber start


KeyError: 'product'

In [32]:
sub_data['product']['id']

KeyError: 'product'

In [25]:
# Функция для бомбардировки запросами в clinton, ничего менять не нужно
def bomber():
    print('bomber start')

    counter = 0

    with requests.Session() as s:
        with open('json1.json', 'r') as json1:
            sub_data = json.load(json1)
            for i in range(ticket_number):
                for account_id in account_list:
                    sub_data['accountId'] = account_id

                    for index, row in product_list.iterrows():
                        
                        sub_data['product']['id'] = index['ID_продукта']
                        sub_data['tariff']['id'] = index['ID_триального_тарифа']
            
                        counter += 1
                        print(sub_data)
                        print('--------------------------------------------------')
                        # s.post(url, data=json.dumps(sub_data), headers=headers)
                        # print('', counter1)

    print('------------------------------------------------')
    print('итого, создано заявок:', counter)
